In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

def extract_bing_links(html):
    soup = BeautifulSoup(html, "lxml")
    results = soup.find_all("li", class_="b_algo")
    links = []
    for result in results:
        a_tag = result.find("a")
        if a_tag and a_tag.has_attr("href"):
            links.append(a_tag["href"])
    return links

def generate_bing_actual_urls(country, city, industry, count=20, max_pages=10):
    query = f"{industry} {city} {country}"
    encoded_query = query.replace(" ", "+")
    websites = set()
    page = 0

    print(f"[+] Searching Bing for: {query}")

    while len(websites) < count and page < max_pages:
        url = f"https://www.bing.com/search?q={encoded_query}&first={page * 10 + 1}"
        print(f"    → Fetching page {page + 1}: {url}")
        try:
            response = requests.get(url, headers=headers)
            if response.status_code != 200:
                print(f"    [!] Failed to fetch page: Status {response.status_code}")
                break

            links = extract_bing_links(response.text)
            prev_count = len(websites)
            for link in links:
                if link not in websites:
                    websites.add(link)
                if len(websites) >= count:
                    break

            if len(websites) == prev_count:
                print("    [!] No new results found. Stopping early.")
                break

        except Exception as e:
            print(f"    [!] Error: {e}")
            break

        page += 1
        time.sleep(1)  # polite delay

    df = pd.DataFrame(list(websites), columns=["website"])
    df["search_query"] = query
    df.to_csv("actual_websites.csv", index=False)
    print(f"[✓] Saved {len(df)} website URLs to actual_websites.csv")

# Example usage
if __name__ == "__main__":
    generate_bing_actual_urls(
        country="USA",
        city="Newyork",
        industry="Jewelery Store",
        count=200,
        max_pages=20
    )
